In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# 加载训练数据
#数据预处理
order_train = pd.read_csv('data/order_train2.csv')

# 将订单数据中的日期从字符串类型转换为日期类型
order_train['order_date'] = pd.to_datetime(order_train['order_date'])
order_train['year'] = order_train['order_date'].dt.year
order_train['month'] = order_train['order_date'].dt.month
order_train['day'] = order_train['order_date'].dt.day
order_train['weekday'] = order_train['order_date'].dt.weekday
order_train['week'] = order_train['order_date'].dt.isocalendar().week
order_train["season"] = ((order_train["order_date"].dt.month-1)//3+1)
# 将季节编码转换为季节名称
# season_names = {1: 0, 2: 1, 3: 2, 4: "冬季"}
# order_train["season"] = order_train["season"].map(season_names)


order_train = order_train[order_train['ord_qty'] < order_train['ord_qty'].quantile(0.99)]  # 清除需求量极大异常值
order_train['sales_chan_name'] = order_train['sales_chan_name'].replace({'online': 0, 'offline': 1})

print(order_train)

predict_df = pd.read_csv('data/predict_sku1.csv')
print('查看是否有重复值')
print(predict_df.duplicated().sum()) #发现没有重复值
print(predict_df.describe())
print(predict_df.info())

#（predict_sku1.csv）提供了产品的销售区域编码、产品编码、产品品类和产品细品类
predict_df = pd.read_csv('data/predict_sku1.csv')
print(predict_df)


       order_date  sales_region_code  item_code  first_cate_code  \
0      2015-09-01                104      22069              307   
1      2015-09-01                104      20028              301   
2      2015-09-02                104      21183              307   
3      2015-09-02                104      20448              308   
4      2015-09-02                104      21565              307   
...           ...                ...        ...              ...   
597377 2018-12-20                102      20994              302   
597378 2018-12-20                102      21875              302   
597379 2018-12-20                102      20215              302   
597380 2018-12-20                102      20195              302   
597381 2018-12-20                102      20321              302   

        second_cate_code  sales_chan_name  item_price  ord_qty  year  month  \
0                    403                1      1114.0       19  2015      9   
1                    405 

In [2]:
from sklearn.model_selection import train_test_split

selected_features = ['sales_region_code','item_code','first_cate_code','second_cate_code','year','month','week','day','weekday','season']
# 划分数据集为训练集和测试集，其中训练集占比为 0.8
X_train, X_test, y_train, y_test = train_test_split(order_train[selected_features], order_train['ord_qty'], test_size=0.2, random_state=42)


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# 定义随机森林模型
model = RandomForestRegressor(random_state=42)

# 定义参数空间
param_grid = {
    'n_estimators': [100,150,200,300],
    'max_depth': [5, 10, 15,20],
    'min_samples_leaf': [2, 4, 6,8],
}

# 进行网格搜索，调整超参数
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

# 输出最佳参数
print(grid_search.best_params_)   # n_estimators=150, max_depth=15 ，min_samples_leaf = 6

In [8]:
# 使用最佳参数进行模型训练
model = RandomForestRegressor(n_estimators=200, max_depth=15,min_samples_leaf = 6, random_state=42)
model.fit(X_train, y_train)


RandomForestRegressor(max_depth=15, min_samples_leaf=6, n_estimators=200,
                      random_state=42)

export_graphviz是一个用于导出决策树的函数，并生成Graphviz格式的文件。该函数接受多个参数作为输入来指定导出的决策树的各种视觉属性。

具体来说，这些参数的含义是:

estimator: 训练好的决策树分类器对象。
out_file: 输出文件的路径或文件名，如果未提供，则代表不将输出写入文件。
feature_names: 特征名称的列表或数组，决策树中每个节点的特征命名时会用到。
rounded: 对于非叶节点的框，进行圆角处理。
proportion: 如果设置为True，则在叶节点中显示的样本比例代替样本数量。
precision: 用于显示double值的小数位数。
filled: 如果设置为True，则使用特征重要性填充节点框的颜色。

In [15]:
#由于模型使用了随机森林，其中每个决策树的结构可能不同，我们可以通过查看单棵决策树来了解模型的预测过程。
from sklearn.tree import export_graphviz
import pydotplus
from IPython.display import Image

# 取出一棵树
estimator = model.estimators_[0]
# 可视化决策树
dot_data = export_graphviz(estimator, out_file=None,
                           feature_names=selected_features,
                           rounded=True, 
                           proportion=False, 
                           precision=2, 
                           filled=True
                           ) 
graph = pydotplus.graph_from_dot_data(dot_data)
Image(graph.create_png())


TypeError: export_graphviz() got an unexpected keyword argument 'dpi'

In [9]:
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from sklearn.metrics import accuracy_score
# 对模型进行预测
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
# 计算准确率
# accuracy = accuracy_score(y_test, y_pred)
# 计算均方误差（MSE）、均方根误差（RMSE）、平均绝对误差（MAE）
print('MSE:', mse)
print('RMSE:',rmse )
print('MAE:', mae)
print('R2:',r2 )
# print(f"Test accuracy = {accuracy:.2f}")
# MSE: 12244.537715590126
# RMSE: 110.65503926884725
# MAE: 66.09853736895502
# R2: 0.2000340063425199
# Test accuracy = 0.01

MSE: 12373.503372027875
RMSE: 111.23625026055075
MAE: 66.6875519437643
R2: 0.19622833305201615


In [ ]:
from sklearn.tree import export_graphviz
import pydotplus
from IPython.display import Image

# 取出一棵树
estimator = model.estimators_[0]

# 可视化决策树
dot_data = export_graphviz(estimator, out_file=None,
                           feature_names=selected_features,
                           rounded=True, 
                           proportion=False, 
                           precision=2, 
                           filled=True)  
graph = pydotplus.graph_from_dot_data(dot_data)
Image(graph.create_png())

In [10]:
date_range = pd.date_range(start='2019-01-01', end='2019-03-31', freq='D')

cols = ['sales_region_code', 'item_code', '2019年1月预测需求量', '2019年2月预测需求量', '2019年3月预测需求量']
predicted_demand_df = pd.DataFrame(columns=cols)

for _, row in predict_df.iterrows():
    predict_data = row.to_frame().transpose()
    region = int(predict_data['sales_region_code'])
    code = int(predict_data['item_code'])
    
    # 扩展日期列
    predict_data = pd.concat([predict_data]*len(date_range), ignore_index=True)
    predict_data['date'] = date_range
    # 提取特征
    predict_data['year'] = predict_data['date'].dt.year
    predict_data['month'] = predict_data['date'].dt.month
    predict_data['week'] = predict_data['order_date'].dt.week
    predict_data['day'] = predict_data['date'].dt.day
    predict_data['weekday'] = predict_data['date'].dt.weekday
    predict_data["season"] = ((predict_data["date"].dt.month-1)//3+1)
    # 进行预测
    X_predict = predict_data[selected_features]
    y_predict = model.predict(X_predict)

    # 计算月总需求量
    predict_data['demand'] = y_predict
    monthly_demand = predict_data.groupby(['year', 'month'], as_index=False)['demand'].sum().astype(int)
    
    row = [region,code, 0, 0, 0]
    for _, row_month in monthly_demand.iterrows():
        month_col = '2019年{}月预测需求量'.format(int(row_month['month']))
        row[cols.index(month_col)] = row_month['demand']     
    predicted_demand_df.loc[len(predicted_demand_df)] = row
    print(predicted_demand_df)
    

#保存
predicted_demand_df.to_excel('data/result1.xlsx', index=False)
print("预测完成")

   sales_region_code  item_code  2019年1月预测需求量  2019年2月预测需求量  2019年3月预测需求量
0                101      20002           301           276           315
   sales_region_code  item_code  2019年1月预测需求量  2019年2月预测需求量  2019年3月预测需求量
0                101      20002           301           276           315
1                101      20003          3686          3422          4176
   sales_region_code  item_code  2019年1月预测需求量  2019年2月预测需求量  2019年3月预测需求量
0                101      20002           301           276           315
1                101      20003          3686          3422          4176
2                101      20006          3157          3357          3569
   sales_region_code  item_code  2019年1月预测需求量  2019年2月预测需求量  2019年3月预测需求量
0                101      20002           301           276           315
1                101      20003          3686          3422          4176
2                101      20006          3157          3357          3569
3                101      20011       

    sales_region_code  item_code  2019年1月预测需求量  2019年2月预测需求量  2019年3月预测需求量
0                 101      20002           301           276           315
1                 101      20003          3686          3422          4176
2                 101      20006          3157          3357          3569
3                 101      20011          1280          1162          1449
4                 101      20014          7730          8768          8012
5                 101      20016          1965          1749          2148
6                 101      20018          5584          4856          5473
7                 101      20020           418           284           317
8                 101      20021           451           296           324
9                 101      20024          3086          1592          1712
10                101      20039           933           728           825
11                101      20041          8044          7800         10061
12                101    

    sales_region_code  item_code  2019年1月预测需求量  2019年2月预测需求量  2019年3月预测需求量
0                 101      20002           301           276           315
1                 101      20003          3686          3422          4176
2                 101      20006          3157          3357          3569
3                 101      20011          1280          1162          1449
4                 101      20014          7730          8768          8012
5                 101      20016          1965          1749          2148
6                 101      20018          5584          4856          5473
7                 101      20020           418           284           317
8                 101      20021           451           296           324
9                 101      20024          3086          1592          1712
10                101      20039           933           728           825
11                101      20041          8044          7800         10061
12                101    

    sales_region_code  item_code  2019年1月预测需求量  2019年2月预测需求量  2019年3月预测需求量
0                 101      20002           301           276           315
1                 101      20003          3686          3422          4176
2                 101      20006          3157          3357          3569
3                 101      20011          1280          1162          1449
4                 101      20014          7730          8768          8012
5                 101      20016          1965          1749          2148
6                 101      20018          5584          4856          5473
7                 101      20020           418           284           317
8                 101      20021           451           296           324
9                 101      20024          3086          1592          1712
10                101      20039           933           728           825
11                101      20041          8044          7800         10061
12                101    

    sales_region_code  item_code  2019年1月预测需求量  2019年2月预测需求量  2019年3月预测需求量
0                 101      20002           301           276           315
1                 101      20003          3686          3422          4176
2                 101      20006          3157          3357          3569
3                 101      20011          1280          1162          1449
4                 101      20014          7730          8768          8012
..                ...        ...           ...           ...           ...
58                101      20193          1515          1344          1383
59                101      20195          1515          1344          1383
60                101      20198          2650          2349          2687
61                101      20206          2627          2230          2501
62                101      20215          1554          1450          1503

[63 rows x 5 columns]
    sales_region_code  item_code  2019年1月预测需求量  2019年2月预测需求量  2019年3月预测需求量
0 

...

      sales_region_code  item_code  2019年1月预测需求量  2019年2月预测需求量  2019年3月预测需求量
0                   101      20002           301           276           315
1                   101      20003          3686          3422          4176
2                   101      20006          3157          3357          3569
3                   101      20011          1280          1162          1449
4                   101      20014          7730          8768          8012
...                 ...        ...           ...           ...           ...
2559                105      21892          2149          1998          2102
2560                105      21893          4307          4202          4851
2561                105      21895          4576          3875          4690
2562                105      21900          5282          4654          5335
2563                105      21902          6543          5610          6372

[2564 rows x 5 columns]
      sales_region_code  item_code  2019年1月预测需求量  2

      sales_region_code  item_code  2019年1月预测需求量  2019年2月预测需求量  2019年3月预测需求量
0                   101      20002           301           276           315
1                   101      20003          3686          3422          4176
2                   101      20006          3157          3357          3569
3                   101      20011          1280          1162          1449
4                   101      20014          7730          8768          8012
...                 ...        ...           ...           ...           ...
2571                105      21923          3655          3304          3660
2572                105      21925           813           720           864
2573                105      21928          3523          1409          1485
2574                105      21942          2065          2937          3523
2575                105      21951          2560          1914          1513

[2576 rows x 5 columns]
      sales_region_code  item_code  2019年1月预测需求量  2

      sales_region_code  item_code  2019年1月预测需求量  2019年2月预测需求量  2019年3月预测需求量
0                   101      20002           301           276           315
1                   101      20003          3686          3422          4176
2                   101      20006          3157          3357          3569
3                   101      20011          1280          1162          1449
4                   101      20014          7730          8768          8012
...                 ...        ...           ...           ...           ...
2583                105      21974          1233          1046          1294
2584                105      21977          1497          1621          1367
2585                105      21978          1471          1599          1343
2586                105      21980          2771          2440          2617
2587                105      21982          2969          2070          1898

[2588 rows x 5 columns]
      sales_region_code  item_code  2019年1月预测需求量  2

      sales_region_code  item_code  2019年1月预测需求量  2019年2月预测需求量  2019年3月预测需求量
0                   101      20002           301           276           315
1                   101      20003          3686          3422          4176
2                   101      20006          3157          3357          3569
3                   101      20011          1280          1162          1449
4                   101      20014          7730          8768          8012
...                 ...        ...           ...           ...           ...
2595                105      22008          1687          1261          1395
2596                105      22009          1514          1045          1253
2597                105      22013          3208          2778          3151
2598                105      22015          2804          2220          3429
2599                105      22021          2708          2076          1673

[2600 rows x 5 columns]
      sales_region_code  item_code  2019年1月预测需求量  2

      sales_region_code  item_code  2019年1月预测需求量  2019年2月预测需求量  2019年3月预测需求量
0                   101      20002           301           276           315
1                   101      20003          3686          3422          4176
2                   101      20006          3157          3357          3569
3                   101      20011          1280          1162          1449
4                   101      20014          7730          8768          8012
...                 ...        ...           ...           ...           ...
2607                105      22046          1821          1190          1004
2608                105      22048          1618          1217          1522
2609                105      22050          1924          1534          1745
2610                105      22052          1071          1049          1224
2611                105      22056          1122          1159          1387

[2612 rows x 5 columns]
      sales_region_code  item_code  2019年1月预测需求量  2

In [75]:
# 加载训练数据
#数据预处理
order_train = pd.read_csv('data/order_train2.csv')

# 将订单数据中的日期从字符串类型转换为日期类型
order_train['order_date'] = pd.to_datetime(order_train['order_date'])
order_train['year'] = order_train['order_date'].dt.year
order_train['month'] = order_train['order_date'].dt.month
order_train['day'] = order_train['order_date'].dt.day
order_train['weekday'] = order_train['order_date'].dt.weekday
order_train['week'] = order_train['week'].dt.week

order_train = order_train[order_train['ord_qty'] < order_train['ord_qty'].quantile(0.99)]  # 清除需求量极大异常值
order_train['sales_chan_name'] = order_train['sales_chan_name'].replace({'online': 0, 'offline': 1})


# 将日期设置为索引
order_train.set_index('order_date', inplace=True)

# 按照每周重新采样
weekly_demand = order_train.groupby(['sales_region_code','item_code','first_cate_code','second_cate_code','year','month']).resample('W')['ord_qty'].sum()

#转换为 DataFrame 方便后面按周预测月需求量
order_train = weekly_demand.reset_index()
print(order_train)

         sales_region_code  item_code  first_cate_code  second_cate_code  \
0                      101      20001              302               408   
1                      101      20001              302               408   
2                      101      20001              302               408   
3                      101      20001              302               408   
4                      101      20002              303               406   
...                    ...        ...              ...               ...   
1167129                105      22084              302               408   
1167130                105      22084              302               408   
1167131                105      22084              302               408   
1167132                105      22084              302               408   
1167133                105      22084              302               408   

         month order_date  ord_qty  
0            3 2016-03-20      102  
1            

In [76]:

from sklearn.model_selection import train_test_split
selected_features = ['sales_region_code','item_code','first_cate_code','second_cate_code','year','month','week']
# 划分数据集为训练集和测试集，其中训练集占比为 0.8
X_train, X_test, y_train, y_test = train_test_split(order_train[selected_features], order_train['ord_qty'], test_size=0.2, random_state=42)


In [77]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# 定义随机森林模型
# model = RandomForestRegressor(random_state=42)

# # 定义参数空间
# param_grid = {
#     'n_estimators': [10, 20, 50,100,150,200],
#     'max_depth': [5, 10, 15,20,25,30]
# }

# # 进行网格搜索，调整超参数
# grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1)
# grid_search.fit(X_train, y_train)

# # 输出最佳参数
# print(grid_search.best_params_)   # n_estimators=150, max_depth=15


# 使用最佳参数进行模型训练
model = RandomForestRegressor(n_estimators=150, max_depth=15, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(max_depth=15, n_estimators=150, random_state=42)

In [78]:
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score

# 对模型进行预测
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
# 计算均方误差（MSE）、均方根误差（RMSE）、平均绝对误差（MAE）
print('MSE:', mse)
print('RMSE:',rmse )
print('MAE:', mean_absolute_error(y_test, y_pred))
print('R2:', r2_score(y_test, y_pred))

MSE: 51051.81285714447
RMSE: 225.94648228539535
MAE: 63.626338734358654
R2: 0.10388971210154485


In [68]:

date_range = pd.date_range(start='2019-01-01', end='2019-03-31', freq='W')

cols = ['sales_region_code', 'item_code', '2019年1月预测需求量', '2019年2月预测需求量', '2019年3月预测需求量']
predicted_demand_df = pd.DataFrame(columns=cols)

#按周的时间粒度去预测
for _, row in predict_df.iterrows():
    predict_data = row.to_frame().transpose()
    region = int(predict_data['sales_region_code'])
    code = int(predict_data['item_code'])
    
    # 扩展日期列
    predict_data = pd.concat([predict_data]*len(date_range), ignore_index=True)
    predict_data['date'] = date_range
    # 提取特征
    predict_data['year'] = predict_data['date'].dt.year
    predict_data['month'] = predict_data['date'].dt.month
    predict_data['day'] = predict_data['date'].dt.day
    predict_data['weekday'] = predict_data['date'].dt.weekday
    
    # 进行预测
    X_predict = predict_data[selected_features]
    y_predict = model.predict(X_predict)
    # 计算月总需求量
    predict_data['demand'] = y_predict*6
    monthly_demand = predict_data.groupby(['year', 'month'], as_index=False)['demand'].sum()
    row = [region,code, 0, 0, 0]
    for _, row_month in monthly_demand.iterrows():
        month_col = '2019年{}月预测需求量'.format(int(row_month['month']))
        row[cols.index(month_col)] = row_month['demand']     
    predicted_demand_df.loc[len(predicted_demand_df)] = row
    
#保存
# predicted_demand_df.to_excel('data/result1.xlsx', index=False)                               

DatetimeIndex(['2019-01-06', '2019-01-13', '2019-01-20', '2019-01-27',
               '2019-02-03', '2019-02-10', '2019-02-17', '2019-02-24',
               '2019-03-03', '2019-03-10', '2019-03-17', '2019-03-24',
               '2019-03-31'],
              dtype='datetime64[ns]', freq='W-SUN')


In [70]:
# 加载训练数据
#数据预处理
order_train = pd.read_csv('data/order_train2.csv')

# 将订单数据中的日期从字符串类型转换为日期类型
order_train['order_date'] = pd.to_datetime(order_train['order_date'])
order_train['year'] = order_train['order_date'].dt.year
order_train['month'] = order_train['order_date'].dt.month
order_train['day'] = order_train['order_date'].dt.day
order_train['weekday'] = order_train['order_date'].dt.weekday


order_train = order_train[order_train['ord_qty'] < order_train['ord_qty'].quantile(0.99)]  # 清除需求量极大异常值
order_train['sales_chan_name'] = order_train['sales_chan_name'].replace({'online': 0, 'offline': 1})


# 将日期设置为索引
order_train.set_index('order_date', inplace=True)
# 按照每周重新采样
month_demand = order_train.groupby(['sales_region_code','item_code','first_cate_code','second_cate_code','year','month']).resample('M')['ord_qty'].sum()
#转换为 DataFrame 方便后面按周预测月需求量
order_train = month_demand.reset_index()
print(order_train)

     sales_region_code order_date  ord_qty
0                  101 2015-09-30   209506
1                  101 2015-10-31   128234
2                  101 2015-11-30   152599
3                  101 2015-12-31   290294
4                  101 2016-01-31   359465
..                 ...        ...      ...
171                105 2018-08-31   290110
172                105 2018-09-30   437179
173                105 2018-10-31   537233
174                105 2018-11-30   328244
175                105 2018-12-31   219519

[176 rows x 3 columns]


In [ ]:
from sklearn.model_selection import train_test_split

selected_features = ['sales_region_code','item_code','first_cate_code','second_cate_code','year','month']
# 划分数据集为训练集和测试集，其中训练集占比为 0.8
X_train, X_test, y_train, y_test = train_test_split(order_train[selected_features], order_train['ord_qty'], test_size=0.2, random_state=42)


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# 定义随机森林模型
# model = RandomForestRegressor(random_state=42)

# # 定义参数空间
# param_grid = {
#     'n_estimators': [10, 20, 50,100,150,200],
#     'max_depth': [5, 10, 15,20,25,30]
# }

# # 进行网格搜索，调整超参数
# grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1)
# grid_search.fit(X_train, y_train)

# # 输出最佳参数
# print(grid_search.best_params_)   # n_estimators=100, max_depth=15


# 使用最佳参数进行模型训练
model = RandomForestRegressor(n_estimators=150, max_depth=15, random_state=42)
model.fit(X_train, y_train)

In [69]:

date_range = pd.date_range(start='2019-01-01', end='2019-03-31', freq='M')
cols = ['sales_region_code', 'item_code', '2019年1月预测需求量', '2019年2月预测需求量', '2019年3月预测需求量']
predicted_demand_df = pd.DataFrame(columns=cols)

#按月的时间粒度去预测
for _, row in predict_df.iterrows():
    predict_data = row.to_frame().transpose()
    region = int(predict_data['sales_region_code'])
    code = int(predict_data['item_code'])
    
    # 扩展日期列
    predict_data = pd.concat([predict_data]*len(date_range), ignore_index=True)
    predict_data['date'] = date_range
    # 提取特征
    predict_data['year'] = predict_data['date'].dt.year
    predict_data['month'] = predict_data['date'].dt.month
    predict_data['day'] = predict_data['date'].dt.day
    predict_data['weekday'] = predict_data['date'].dt.weekday
    
    # 进行预测
    X_predict = predict_data[selected_features]
    y_predict = model.predict(X_predict)
    # 计算月总需求量
    predict_data['demand'] = y_predict*6
    monthly_demand = predict_data.groupby(['year', 'month'], as_index=False)['demand'].sum()
    row = [region,code, 0, 0, 0]
    for _, row_month in monthly_demand.iterrows():
        month_col = '2019年{}月预测需求量'.format(int(row_month['month']))
        row[cols.index(month_col)] = row_month['demand']     
    predicted_demand_df.loc[len(predicted_demand_df)] = row
#保存
# predicted_demand_df.to_excel('data/result1.xlsx', index=False)                               

DatetimeIndex(['2019-01-31', '2019-02-28', '2019-03-31'], dtype='datetime64[ns]', freq='M')
